In [286]:
import sys
import timeit
from heapq import heappop, heappush

In [287]:
args = sys.argv[1:]

In [288]:
# exit if user does not enter two parameters
if len(args) != 2:
    print("ERROR: Not enough or too many input arguments.")
    exit()


In [289]:
# Defining the initial and goal state 
INITIAL = "NM" #Initial State
GOAL = "MS" #Goal State

In [290]:
# Loading the csv files
driving = open('/content/driving.csv')
straight = open('/content/straightline.csv')

In [291]:
# dictionary initialization 
state_neighbors = {}
straight_distances = {}

In [292]:
# enumerating list data
driving_enum = enumerate(driving)
straight_enum = enumerate(straight)

In [293]:
# Node class which hold all parents node and its cost
class Node:
    def __init__(self, state, parent, pathCost, heuristics, algorithm):
        self.STATE = state
        self.PARENT = parent
        self.COST = pathCost
        self.NEIGHBORS = {}
        self.HEURISTICS = heuristics
        if algorithm == 'greedy_best_first_search':
            self.EVAL = self.HEURISTICS
        elif algorithm == 'a_star_algorithm':
            self.EVAL = self.COST + self.HEURISTICS
        
    def getState(self):
        return self.STATE
        
    def getParent(self):
        return self.PARENT

    def getPathCost(self):
        return self.PATHCOST
        
    def getHeuristics(self):
        return self.HEURISTICS
      
    def getEval(self):
        return self.EVAL
      
    def __lt__(self, other):
        return self.getEval() < other.getEval()

In [294]:
# Fills two maps passed as inputs. State name to TreeNode and state name to distance to the target.
# The third input is the target name

stOrder = None
for index, row in driving_enum:
    if index == 0:
        # splitting data
        stOrder = row.split(",")[1:]
        stOrder[-1] = stOrder[-1][:-1]
        continue
    start = stOrder[index - 1]
    state_neighbors[start] = {}
    data = row.split(",")
    data[-1] = data[-1][:-1]
    for j in range(1, len(data)):
        value = int(data[j])
        if value in [0, -1]:
            continue
        state_neighbors[start][stOrder[j - 1]] = value
driving.close()


In [295]:
state_neighbors

{'AL': {'FL': 210, 'GA': 160, 'MS': 248, 'TN': 281},
 'AR': {'LA': 344, 'MO': 345, 'MS': 263, 'OK': 340, 'TN': 349, 'TX': 514},
 'AZ': {'CA': 755, 'NM': 480, 'NV': 733, 'UT': 659},
 'CA': {'AZ': 755, 'NV': 131, 'OR': 535},
 'CO': {'KS': 541, 'NE': 487, 'NM': 392, 'OK': 679, 'UT': 521, 'WY': 102},
 'CT': {'MA': 102, 'NY': 114, 'RI': 87},
 'DC': {'MD': 35, 'VA': 106},
 'DE': {'MD': 64, 'NJ': 112, 'PA': 129},
 'FL': {'AL': 210, 'GA': 260},
 'GA': {'AL': 160, 'FL': 260, 'NC': 260, 'SC': 214, 'TN': 250},
 'IA': {'IL': 335, 'MN': 245, 'MO': 266, 'NE': 187, 'SD': 503, 'WI': 293},
 'ID': {'MT': 480, 'NV': 449, 'OR': 476, 'UT': 344, 'WA': 534, 'WY': 737},
 'IL': {'IA': 335, 'IN': 211, 'KY': 373, 'MO': 195, 'WI': 263},
 'IN': {'IL': 211, 'KY': 164, 'MI': 254, 'OH': 173},
 'KS': {'CO': 541, 'MO': 220, 'NE': 165, 'OK': 293},
 'KY': {'IL': 373,
  'IN': 164,
  'MO': 446,
  'OH': 192,
  'TN': 208,
  'VA': 513,
  'WV': 198},
 'LA': {'AR': 344, 'MS': 623, 'TX': 1067},
 'MA': {'CT': 104, 'NH': 68, 'NY':

In [296]:
for index, row in straight_enum:
    if index == 0 or index > len(stOrder):
        continue
    data = row.split(",")
    data[-1] = data[-1][:-1]
    start = stOrder[index - 1]
    for j in range(1, len(stOrder) + 1):
        if start == GOAL:
            straight_distances[stOrder[j - 1]] = int(data[j])
straight.close()


In [297]:
straight_distances

{'AL': 227,
 'AR': 208,
 'AZ': 1279,
 'CA': 1806,
 'CO': 972,
 'CT': 1162,
 'DC': 867,
 'DE': 947,
 'FL': 371,
 'GA': 350,
 'IA': 670,
 'ID': 1609,
 'IL': 519,
 'IN': 563,
 'KS': 559,
 'KY': 506,
 'LA': 141,
 'MA': 1255,
 'MD': 895,
 'ME': 1377,
 'MI': 784,
 'MN': 834,
 'MO': 448,
 'MS': 0,
 'MT': 1519,
 'NC': 703,
 'ND': 1148,
 'NE': 690,
 'NH': 1262,
 'NJ': 1017,
 'NM': 931,
 'NV': 1715,
 'NY': 1147,
 'OH': 664,
 'OK': 474,
 'OR': 1959,
 'PA': 921,
 'RI': 1222,
 'SC': 542,
 'SD': 998,
 'TN': 330,
 'TX': 468,
 'UT': 1334,
 'VA': 807,
 'VT': 1285,
 'WA': 1995,
 'WI': 746,
 'WV': 638,
 'WY': 1013}

In [298]:
def greedy_best_first_search():
    # Starting time
    timeStart = timeit.default_timer()
    state = INITIAL
    # Creating initial node
    node = Node(state, None, float("inf"), 0, "greedy_best_first_search")
    node.NEIGHBORS = state_neighbors[INITIAL]
    node.COST = 0
    try:
        frtr = [[straight_distances[INITIAL],INITIAL,node]]
        reached_state = {INITIAL:node}
        finished = False
        while frtr:
            node = heappop(frtr)[2]
            if node.STATE == GOAL:
                finished = True
                break
            for neighbor in node.NEIGHBORS:
                if neighbor not in reached_state or node.COST + node.NEIGHBORS[neighbor] < reached_state[neighbor].COST:
                    if neighbor not in reached_state:
                        reached_state[neighbor] = Node(neighbor, None, float("inf"), 0, "greedy_best_first_search")
                        reached_state[neighbor].COST = node.COST + node.NEIGHBORS[neighbor]
                        reached_state[neighbor].NEIGHBORS = state_neighbors[neighbor]
                    reached_state[neighbor].COST = node.COST + node.NEIGHBORS[neighbor]
                    reached_state[neighbor].PARENT = node
                    heappush(frtr, [straight_distances[neighbor],neighbor,reached_state[neighbor]])
                    
        timeEnd = timeit.default_timer()
        totalCost = node.COST
        path_string = node.STATE
        while node.PARENT:
            node = node.PARENT
            path_string = node.STATE+" "+path_string

        path_list = path_string.split(' ')

        return "Greedy Best First: \n"+ "Initial: "+INITIAL+" | Goal: "+GOAL+" | Path: "+str(path_list)+"\nPath cost: "+str(totalCost)+" miles\nExecution time: "+str(timeEnd-timeStart)+" seconds"
    except:
        timeEnd = timeit.default_timer()
        return "Path not found / failure\nGreedy Best First: \n"+ "Initial: "+INITIAL+" | Goal: "+GOAL+" | Path: [NOT FOUND] \nPath cost: N/A miles\nExecution time: "+str(timeEnd-timeStart)+" seconds"


In [299]:
def a_star_algorithm():
    timeStart = timeit.default_timer()
    state = INITIAL
    node = Node(state, None, float("inf"), 0, "a_star_algorithm")
    node.NEIGHBORS = state_neighbors[INITIAL]
    node.COST = 0
    try:
        frtr = [[straight_distances[INITIAL],INITIAL,node]]
        reached_state = {INITIAL:node}
        finished = False
        while frtr:
            node = heappop(frtr)[2]
            if node.STATE == GOAL:
                finished = True
                break
            for neighbor in node.NEIGHBORS:
                if neighbor not in reached_state or node.COST + node.NEIGHBORS[neighbor] < reached_state[neighbor].COST:
                    if neighbor not in reached_state:
                        reached_state[neighbor] = Node(neighbor, None, float("inf"), 0, "greedy_best_first_search")
                        reached_state[neighbor].COST = node.COST + node.NEIGHBORS[neighbor]
                        reached_state[neighbor].NEIGHBORS = state_neighbors[neighbor]
                    reached_state[neighbor].COST = node.COST + node.NEIGHBORS[neighbor]
                    reached_state[neighbor].PARENT = node
                    heappush(frtr, [reached_state[neighbor].COST+straight_distances[neighbor],neighbor,reached_state[neighbor]]) # a_star_algorithm
                    
        timeEnd = timeit.default_timer()
        totalCost = node.COST
        path_string = node.STATE
        while node.PARENT:
            node = node.PARENT
            path_string = node.STATE+" "+path_string

        path_list = path_string.split(' ')


        return "A* : \n"+ "Initial: "+INITIAL+" | Goal: "+GOAL+" | Path: "+str(path_list)+"\nPath cost: "+str(totalCost)+" miles\nExecution time: "+str(timeEnd-timeStart)+" seconds"
    except:
        timeEnd = timeit.default_timer()
        return "Path not found / failure\nA*: \n"+ "Initial: "+INITIAL+" | Goal: "+GOAL+" | Path: [NOT FOUND] \nPath cost: N/A miles\nExecution time: "+str(timeEnd-timeStart)+" seconds"


In [302]:
print("SINGH, AMAN, A20491333 SOLUTION:\n")
print(greedy_best_first_search()+"\n \n"+a_star_algorithm())

SINGH, AMAN, A20491333 SOLUTION:

Greedy Best First: 
Initial: NM | Goal: MS | Path: ['NM', 'TX', 'LA', 'MS']
Path cost: 2378 miles
Execution time: 4.14120004279539e-05 seconds
 
A* : 
Initial: NM | Goal: MS | Path: ['NM', 'OK', 'AR', 'MS']
Path cost: 1137 miles
Execution time: 3.408600059628952e-05 seconds
